In [10]:
import numpy as np
import random
import matplotlib.pyplot as plt
import time
import networkx as nx
import sys
import os
import copy
import datetime
import json
from joblib import Parallel, delayed
import CSD

import importlib
importlib.reload(CSD)
#%matplotlib inline

<module 'CSD' from 'C:\\Users\\rtt-lab\\Desktop\\tsubasa\\conv_const\\CSD.py'>

# Setting

In [14]:
num_node_array = [49]
num_depot_array = [8]
num_driver_array = [64]

net_per_numnode = 1
times_per_net = 10

logit_param_q = 5 # 1 done
logit_param_n = 5

# Generate Data

In [15]:
for num_node in num_node_array:
    
    for num_depot in num_depot_array:

        for num_per_driver in num_driver_array:
            
            num_sum_driver = num_per_driver*num_node*num_node

            data = CSD.DataMaker(num_node, num_depot, num_sum_driver, logit_param_q, logit_param_n)

            for netnum in range(net_per_numnode): #net_numberは0以上net_per_numnode未満
                #inputフォルダの作成
                path_net = r"input/num_driver_{nv}/node_{nn}/depot_{nd}/net_{net}/network".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum)
                path_dep = r"input/num_driver_{nv}/node_{nn}/depot_{nd}/net_{net}/deplist".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum)
                path_dist = r"input/num_driver_{nv}/node_{nn}/depot_{nd}/net_{net}/dist".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum)
                path_cost = r"input/num_driver_{nv}/node_{nn}/depot_{nd}/net_{net}/cost".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum)
                path_cost_atom = r"input/num_driver_{nv}/node_{nn}/depot_{nd}/net_{net}/cost_atomic".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum)
                path_demand = r"input/num_driver_{nv}/node_{nn}/depot_{nd}/net_{net}/demand".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum)
                os.makedirs(path_net, exist_ok=True)
                os.makedirs(path_dep, exist_ok=True)
                os.makedirs(path_dist, exist_ok=True)
                os.makedirs(path_cost, exist_ok=True)
                os.makedirs(path_cost_atom, exist_ok=True)
                os.makedirs(path_demand, exist_ok=True)

                #ネットワークの作成
                data.make_network()
                data.save_network(path_net + r"/network_{nn}_{nd}_{net}.json".format(nn=num_node, nd=num_depot, net=netnum))

                #タスク起点のリストを作成
                data.make_depot_list()
                data.save_depot_list(path_dep + r"/deplist_{nn}_{nd}_{net}.csv".format(nn=num_node, nd=num_depot, net=netnum))

                #各起終点の距離(dijkstra法を用いて)行列を作成
                data.make_distmat()
                data.save_distmat(path_dist + r"/dist_{nn}_{nd}_{net}.csv".format(nn=num_node, nd=num_depot, net=netnum))

                #ドライバーのタスク実行コスト行列を作成
                data.make_costmat()
                data.save_costmat(path_cost + r"/cost_{nn}_{nd}_{net}.csv".format(nn=num_node, nd=num_depot, net=netnum))

                for exp_num in range(times_per_net):

                    #各ODペアのドライバー数，タスク数をランダムで生成
                    data.make_demand_supply()
                    data.save_demand_supply(path_demand + r"/demand_{nn}_{nd}_{net}_{exp}.csv".format(nn=num_node, nd=num_depot, net=netnum, exp=exp_num))

                    #個人の費用行列を生成
                    data.make_atomic_costmat()
                    data.save_atomic_costmat(path_cost_atom + r"/cost_atomic_driver_{nn}_{nd}_{net}_{exp}.csv".format(nn=num_node, nd=num_depot, net=netnum, exp=exp_num),path_cost_atom + r"/cost_atomic_task_{nn}_{nd}_{net}_{exp}.csv".format(nn=num_node, nd=num_depot, net=netnum, exp=exp_num))

                    comp = "COMPLETE  num_driver:{nv}, num_node:{nn}, num_depot:{nd}, network_num:{net}, experience_num:{exp}\n".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=exp_num)
                    print(comp)
                
                print("\n")
            print("\n")
        print("\n")
    print("\n")
print("END")

completed making a network
completed making the distance matrix
completed making o-r-s-d costs
COMPLETE  num_driver:153664, num_node:49, num_depot:8, network_num:0, experience_num:0

COMPLETE  num_driver:153664, num_node:49, num_depot:8, network_num:0, experience_num:1

COMPLETE  num_driver:153664, num_node:49, num_depot:8, network_num:0, experience_num:2

COMPLETE  num_driver:153664, num_node:49, num_depot:8, network_num:0, experience_num:3

COMPLETE  num_driver:153664, num_node:49, num_depot:8, network_num:0, experience_num:4

COMPLETE  num_driver:153664, num_node:49, num_depot:8, network_num:0, experience_num:5

COMPLETE  num_driver:153664, num_node:49, num_depot:8, network_num:0, experience_num:6

COMPLETE  num_driver:153664, num_node:49, num_depot:8, network_num:0, experience_num:7

COMPLETE  num_driver:153664, num_node:49, num_depot:8, network_num:0, experience_num:8

COMPLETE  num_driver:153664, num_node:49, num_depot:8, network_num:0, experience_num:9









END


# Run Accel Gradient

In [16]:
for num_node in num_node_array:
    
    for num_depot in num_depot_array:

        for num_per_driver in num_driver_array:
            
            num_sum_driver = num_per_driver*num_node*num_node
            

            data_array = [[CSD.DataMaker(num_node, num_depot, num_sum_driver, logit_param_q, logit_param_n) for i in range(times_per_net)] for j in range(net_per_numnode)]
            csd_array = [[None for i in range(times_per_net)] for j in range(net_per_numnode)]

            accel_time_array = np.zeros([net_per_numnode, times_per_net])
            accel_obj_array = np.zeros([net_per_numnode, times_per_net])
            #accel_time_array = [[None for i in range(times_per_net)] for j in range(net_per_numnode)]
            ##LP_max_time_array = [[None for i in range(times_per_net)] for j in range(net_per_numnode)]
            ##LP_sum_time_array = [[None for i in range(times_per_net)] for j in range(net_per_numnode)]
            ##total_time_array = [[None for i in range(times_per_net)] for j in range(net_per_numnode)]
            ##parallel_time_array = [[None for i in range(times_per_net)] for j in range(net_per_numnode)]
            #accel_obj_array = [[None for i in range(times_per_net)] for j in range(net_per_numnode)]
            ##final_obj_array = [[None for i in range(times_per_net)] for j in range(net_per_numnode)]

            path_obj = r"result/result_proposed/num_driver_{nv}/node_{nn}/depot_{nd}/obj".format(nv=num_sum_driver, nn=num_node, nd=num_depot)
            path_time = r"result/result_proposed/num_driver_{nv}/node_{nn}/depot_{nd}/time".format(nv=num_sum_driver, nn=num_node, nd=num_depot)

            os.makedirs(path_obj, exist_ok=True)
            os.makedirs(path_time, exist_ok=True)

            for netnum in range(net_per_numnode):
                #タスク起点ノードの情報と各ノード間のコスト行列を入手
                deplist = np.loadtxt(r"input/num_driver_{nv}/node_{nn}/depot_{nd}/net_{net}/deplist/deplist_{nn}_{nd}_{net}.csv".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum), delimiter=",")
                distmat = np.loadtxt(r"input/num_driver_{nv}/node_{nn}/depot_{nd}/net_{net}/dist/dist_{nn}_{nd}_{net}.csv".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum), delimiter=",")




                for expnum in range(times_per_net):

                    path_alloc = r"result/result_proposed/num_driver_{nv}/node_{nn}/depot_{nd}/net_{net}/allocation/allocation_{exp}".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum)
                    path_alloc_od = r"result/result_proposed/num_driver_{nv}/node_{nn}/depot_{nd}/net_{net}/allocation/allocation_{exp}/allocation_od".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum)
                    path_alloc_rs = r"result/result_proposed/num_driver_{nv}/node_{nn}/depot_{nd}/net_{net}/allocation/allocation_{exp}/allocation_rs".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum)
                    #path_time_exp = r"result/result_proposed/num_driver_{nv}/node_{nn}/depot_{nd}/net_{net}/time/time_{exp}".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum)
                    path_price = r"result/result_proposed/num_driver_{nv}/node_{nn}/depot_{nd}/net_{net}/price/price_{exp}".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum)

                    os.makedirs(path_alloc, exist_ok=True)
                    os.makedirs(path_alloc_od, exist_ok=True)
                    os.makedirs(path_alloc_rs, exist_ok=True)
                    #os.makedirs(path_time_exp, exist_ok=True)
                    os.makedirs(path_price, exist_ok=True)


                    demand = np.loadtxt(r"input/num_driver_{nv}/node_{nn}/depot_{nd}/net_{net}/demand/demand_{nn}_{nd}_{net}_{exp}.csv".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum), delimiter=",").reshape([-1,1])
                    num_drv = demand[:(num_node)**2, :]
                    num_task = demand[(num_node)**2:, :]

                    data_array[netnum][expnum].load_depot_list(deplist.astype(int))
                    data_array[netnum][expnum].load_distmat(distmat)
                    data_array[netnum][expnum].load_demand_supply(num_drv, num_task)


                    csd_array[netnum][expnum] = data_array[netnum][expnum].create_CSD()



                    #加速勾配法の実行
                    csd_array[netnum][expnum].solve_dual()


                    csd_array[netnum][expnum].allocate_tasks()
                    #print(csd_array[netnum][expnum].real_allocation_q.shape, csd_array[netnum][expnum].real_allocation_n.shape)
                    csd_array[netnum][expnum].rounded_allocation()

                    csd_array[netnum][expnum].save_real_allocation(path_alloc + "/real_driver_allocation_{nn}_{nd}_{net}_{exp}.csv".format(nn=num_node, nd=num_depot, net=netnum, exp=expnum), path_alloc + "/real_task_allocation_{nn}_{nd}_{net}_{exp}.csv".format(nn=num_node, nd=num_depot, net=netnum, exp=expnum))

                    csd_array[netnum][expnum].save_rounded_allocation(path_alloc + 
                                                                      "/rounded_driver_allocation_{nn}_{nd}_{net}_{exp}.csv".format(nn=num_node, nd=num_depot, net=netnum, exp=expnum), path_alloc + 
                                                                      "/rounded_task_allocation_{nn}_{nd}_{net}_{exp}.csv".format(nn=num_node, nd=num_depot, net=netnum, exp=expnum))

                    np.savetxt(path_price + r"/price_{nn}_{nd}_{net}_{exp}.csv".format(nn=num_node, nd=num_depot, net=netnum, exp=expnum), csd_array[netnum][expnum].opt_price, delimiter=",")


                    #driver_atomic_cost, task_atomic_cost, num_driver, num_task, allocation_driver, allocation_task = LP.read_data(num_sum_driver, num_node, num_depot, netnum, expnum)

                    obj = 0

                    accel_time_array[netnum][expnum] = csd_array[netnum][expnum].time_for_solve_accel
                    #LP_max_time_array[netnum][expnum] = np.amax(LP_time)
                    #LP_sum_time_array[netnum][expnum] = np.sum(LP_time)
                    #parallel_time_array[netnum][expnum] = csd_array[netnum][expnum].time_for_solve_accel + LP_max_time_array[netnum][expnum]
                    #total_time_array[netnum][expnum] = csd_array[netnum][expnum].time_for_solve_accel + LP_sum_time_array[netnum][expnum]
                    accel_obj_array[netnum][expnum] = csd_array[netnum][expnum].opt_welfare
                    #final_obj_array[netnum][expnum] = obj

                    print("complete_{nv}_{nn}_{nd}_{net}_{exp}".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum))

                    np.savetxt(path_obj + r"/accel_obj_{nv}_{nn}_{nd}.csv".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum), accel_obj_array, delimiter=",")
                    #np.savetxt(path_obj + r"/final_obj_{nv}{nn}_{nd}.csv".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum), final_obj_array, delimiter=",")
                    np.savetxt(path_time + r"/accel_time_{nv}_{nn}_{nd}.csv".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum), accel_time_array, delimiter=",")
                    #np.savetxt(path_time + r"/LP_max_time_{nv}_{nn}_{nd}.csv".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum), LP_max_time_array, delimiter=",")
                    #np.savetxt(path_time + r"/LP_sum_time_{nv}_{nn}_{nd}.csv".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum), LP_sum_time_array, delimiter=",")
                    #np.savetxt(path_time + r"/parallell_time_{nv}_{nn}_{nd}.csv".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum), parallel_time_array, delimiter=",")
                    #np.savetxt(path_time + r"/total_time_{nv}_{nn}_{nd}.csv".format(nv=num_sum_driver, nn=num_node, nd=num_depot, net=netnum, exp=expnum), total_time_array, delimiter=",")

completed loading the distance matrix
processing... 100 objective function: 3964768.0648135333 conv: 316194.03516302863 L: 3298.96902959206
processing... 200 objective function: 4306004.572740367 conv: 142883.99974647778 L: 3298.96902959206
processing... 300 objective function: 4464585.207789688 conv: 135437.05993132683 L: 3298.96902959206
processing... 400 objective function: 4537643.385558287 conv: 38811.49761917011 L: 3298.96902959206
processing... 500 objective function: 4571185.706421756 conv: 32682.64123828298 L: 3298.96902959206
processing... 600 objective function: 4586866.4899304975 conv: 23463.726791841127 L: 3298.96902959206
processing... 700 objective function: 4593818.8491580505 conv: 18768.999999999993 L: 3298.96902959206
processing... 800 objective function: 4596881.225207889 conv: 18768.99999999018 L: 3298.96902959206
processing... 900 objective function: 4598974.451122468 conv: 18759.61514768843 L: 3298.96902959206
processing... 1000 objective function: 4600436.6578410

processing... 8300 objective function: 4605629.792605627 conv: 24.999999999999325 L: 3298.96902959206
processing... 8400 objective function: 4605631.0355071 conv: 24.99999999999856 L: 3298.96902959206
processing... 8500 objective function: 4605632.278361922 conv: 24.99999999999692 L: 3298.96902959206
processing... 8600 objective function: 4605633.5212014625 conv: 24.999999999993427 L: 3298.96902959206
processing... 8700 objective function: 4605634.764033075 conv: 24.999999999985985 L: 3298.96902959206
processing... 8800 objective function: 4605636.006859447 conv: 24.999999999970097 L: 3298.96902959206
processing... 8900 objective function: 4605637.249682076 conv: 24.999999999936186 L: 3298.96902959206
processing... 9000 objective function: 4605638.4925019825 conv: 24.99999999986386 L: 3298.96902959206
processing... 9100 objective function: 4605639.735319899 conv: 24.99999999970954 L: 3298.96902959206
processing... 9200 objective function: 4605640.9781363625 conv: 24.999999999380265 L: 

processing... 16600 objective function: 4605701.079645167 conv: 15.9999999994109 L: 3298.96902959206
processing... 16700 objective function: 4605701.564645173 conv: 15.999999998919858 L: 3298.96902959206
processing... 16800 objective function: 4605702.049645179 conv: 15.999999998019515 L: 3298.96902959206
processing... 16900 objective function: 4605702.534645185 conv: 15.999999996368697 L: 3298.96902959206
processing... 17000 objective function: 4605703.019645191 conv: 15.999999993341849 L: 3298.96902959206
processing... 17100 objective function: 4605703.504645198 conv: 15.999999987791995 L: 3298.96902959206
processing... 17200 objective function: 4605703.989645204 conv: 15.999999977616099 L: 3298.96902959206
processing... 17300 objective function: 4605704.474645209 conv: 15.99999995895816 L: 3298.96902959206
processing... 17400 objective function: 4605704.959645213 conv: 15.999999924748032 L: 3298.96902959206
processing... 17500 objective function: 4605705.444645216 conv: 15.999999862

processing... 3900 objective function: 4503786.980719216 conv: 624.8557801900034 L: 3991.7525258063934
processing... 4000 objective function: 4503825.669801322 conv: 621.7108298815874 L: 3991.7525258063934
processing... 4100 objective function: 4503861.1658150805 conv: 556.579312188362 L: 3991.7525258063934
processing... 4200 objective function: 4503890.467351623 conv: 255.9999999999995 L: 3991.7525258063934
processing... 4300 objective function: 4503911.501595325 conv: 255.99999999999613 L: 3991.7525258063934
processing... 4400 objective function: 4503931.304872407 conv: 255.9999999999713 L: 3991.7525258063934
processing... 4500 objective function: 4503951.025941897 conv: 255.999999999787 L: 3991.7525258063934
processing... 4600 objective function: 4503970.687795572 conv: 255.99999999841992 L: 3991.7525258063934
processing... 4700 objective function: 4503990.103857523 conv: 255.99999998827855 L: 3991.7525258063934
processing... 4800 objective function: 4504008.482255043 conv: 255.9999

processing... 12200 objective function: 4504350.48484871 conv: 56.462526088712046 L: 3991.7525258063934
processing... 12300 objective function: 4504351.666221868 conv: 22.644789332265347 L: 3991.7525258063934
processing... 12400 objective function: 4504352.1726201065 conv: 4.880059733164744 L: 3991.7525258063934
processing... 12500 objective function: 4504352.429275863 conv: 4.0 L: 3991.7525258063934
processing... 12600 objective function: 4504352.637369546 conv: 4.0 L: 3991.7525258063934
processing... 12700 objective function: 4504352.8387486525 conv: 4.0 L: 3991.7525258063934
processing... 12800 objective function: 4504353.039299959 conv: 4.0 L: 3991.7525258063934
processing... 12900 objective function: 4504353.239745719 conv: 4.0 L: 3991.7525258063934
processing... 13000 objective function: 4504353.440174089 conv: 4.0 L: 3991.7525258063934
processing... 13100 objective function: 4504353.640597282 conv: 4.0 L: 3991.7525258063934
processing... 13200 objective function: 4504353.8410177

...

processing... 6000 objective function: 4557812.343680726 conv: 100.0 L: 3628.8659325512663
processing... 6100 objective function: 4557817.328134785 conv: 99.99999999999997 L: 3628.8659325512663
processing... 6200 objective function: 4557822.305920361 conv: 100.0 L: 3628.8659325512663
processing... 6300 objective function: 4557827.263906197 conv: 99.99999999999997 L: 3628.8659325512663
processing... 6400 objective function: 4557832.164676968 conv: 99.99999999999997 L: 3628.8659325512663
processing... 6500 objective function: 4557836.912282223 conv: 99.99999999999997 L: 3628.8659325512663
processing... 6600 objective function: 4557841.32159675 conv: 100.0 L: 3628.8659325512663
processing... 6700 objective function: 4557845.224536791 conv: 100.0 L: 3628.8659325512663
processing... 6800 objective function: 4557848.702975489 conv: 99.99999999999997 L: 3628.8659325512663
processing... 6900 objective function: 4557851.99132686 conv: 100.0 L: 3628.8659325512663
processing... 7000 objective fun

processing... 14700 objective function: 4557965.220098576 conv: 4.0 L: 3628.8659325512663
processing... 14800 objective function: 4557965.688559051 conv: 4.0 L: 3628.8659325512663
processing... 14900 objective function: 4557966.157017799 conv: 4.0 L: 3628.8659325512663
processing... 15000 objective function: 4557966.625474275 conv: 4.0 L: 3628.8659325512663
processing... 15100 objective function: 4557967.093927755 conv: 4.0 L: 3628.8659325512663
processing... 15200 objective function: 4557967.562377291 conv: 4.0 L: 3628.8659325512663
processing... 15300 objective function: 4557968.030821629 conv: 4.0 L: 3628.8659325512663
processing... 15400 objective function: 4557968.4992591245 conv: 4.0 L: 3628.8659325512663
processing... 15500 objective function: 4557968.967687605 conv: 4.0 L: 3628.8659325512663
processing... 15600 objective function: 4557969.436104212 conv: 4.0 L: 3628.8659325512663
processing... 15700 objective function: 4557969.904505181 conv: 4.0 L: 3628.8659325512663
processin

processing... 23800 objective function: 4557998.460851865 conv: 4.0 L: 3628.8659325512663
processing... 23900 objective function: 4557998.708863299 conv: 4.0 L: 3628.8659325512663
processing... 24000 objective function: 4557998.956874705 conv: 4.0 L: 3628.8659325512663
processing... 24100 objective function: 4557999.204886095 conv: 4.0 L: 3628.8659325512663
processing... 24200 objective function: 4557999.452897474 conv: 4.0 L: 3628.8659325512663
processing... 24300 objective function: 4557999.700908849 conv: 4.0 L: 3628.8659325512663
processing... 24400 objective function: 4557999.94892022 conv: 4.0 L: 3628.8659325512663
processing... 24500 objective function: 4558000.19693159 conv: 4.0 L: 3628.8659325512663
processing... 24600 objective function: 4558000.444942959 conv: 4.0 L: 3628.8659325512663
processing... 24700 objective function: 4558000.692954326 conv: 3.999999999999999 L: 3628.8659325512663
processing... 24800 objective function: 4558000.940965693 conv: 4.0 L: 3628.865932551266

processing... 32700 objective function: 4558020.533861879 conv: 3.999999999999999 L: 3628.8659325512663
processing... 32800 objective function: 4558020.781872676 conv: 3.999999999999999 L: 3628.8659325512663
processing... 32900 objective function: 4558021.029883293 conv: 3.9999999999999982 L: 3628.8659325512663
processing... 33000 objective function: 4558021.277893672 conv: 3.9999999999999973 L: 3628.8659325512663
processing... 33100 objective function: 4558021.525903736 conv: 3.9999999999999973 L: 3628.8659325512663
processing... 33200 objective function: 4558021.773913388 conv: 3.9999999999999956 L: 3628.8659325512663
processing... 33300 objective function: 4558022.021922495 conv: 3.9999999999999947 L: 3628.8659325512663
processing... 33400 objective function: 4558022.269930886 conv: 3.999999999999993 L: 3628.8659325512663
processing... 33500 objective function: 4558022.517938334 conv: 3.999999999999991 L: 3628.8659325512663
processing... 33600 objective function: 4558022.765944537 c

processing... 40600 objective function: 4558036.236453606 conv: 3.999997204536463 L: 3628.8659325512663
processing... 40700 objective function: 4558036.374237626 conv: 3.999996317595996 L: 3628.8659325512663
processing... 40800 objective function: 4558036.512021612 conv: 3.9999951492485053 L: 3628.8659325512663
processing... 40900 objective function: 4558036.649805559 conv: 3.9999936102098115 L: 3628.8659325512663
processing... 41000 objective function: 4558036.787589454 conv: 3.9999915828679966 L: 3628.8659325512663
processing... 41100 objective function: 4558036.925373284 conv: 3.9999889122957892 L: 3628.8659325512663
processing... 41200 objective function: 4558037.063157028 conv: 3.9999853944115027 L: 3628.8659325512663
processing... 41300 objective function: 4558037.200940658 conv: 3.99998076038392 L: 3628.8659325512663
processing... 41400 objective function: 4558037.338724144 conv: 3.999974656089587 L: 3628.8659325512663
processing... 41500 objective function: 4558037.476507433 co